In [1]:
# train3에서 deeplearning


In [21]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [59]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from keras import metrics

import tensorflow_addons.metrics

In [23]:
import warnings

# 경고 무시
warnings.filterwarnings("ignore", category=FutureWarning)

In [24]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [25]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
train = train.drop(['user_id'], axis=1)
test_id = test['user_id'].values
test = test.drop(['user_id'], axis=1)



In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  object 
 10  subscription_type                  10000 non-null  object 
 11  customer_inquiry_history           10000 non-null  int6

In [27]:
# preferred_difficulty_level 열 인코딩
train.loc[train['preferred_difficulty_level'].values == 'Low','preferred_difficulty_level'] = 0
train.loc[train['preferred_difficulty_level'].values == 'Medium','preferred_difficulty_level'] = 1
train.loc[train['preferred_difficulty_level'].values == 'High','preferred_difficulty_level'] = 2

test.loc[test['preferred_difficulty_level'].values == 'Low','preferred_difficulty_level'] = 0
test.loc[test['preferred_difficulty_level'].values == 'Medium','preferred_difficulty_level'] = 1
test.loc[test['preferred_difficulty_level'].values == 'High','preferred_difficulty_level'] = 2
train['preferred_difficulty_level'] = train['preferred_difficulty_level'].astype(int)

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  int32  
 10  subscription_type                  10000 non-null  object 
 11  customer_inquiry_history           10000 non-null  int6

In [29]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 데이터 스케일링을 위한 StandardScaler 인스턴스 생성
scaler = StandardScaler()
categoric_col =['subscription_type']
label_encoders = {}
for col in categoric_col:
    label_encoders[col] = LabelEncoder()
    train[col] = label_encoders[col].fit_transform(train[col])
    test[col] = label_encoders[col].fit_transform(test[col])

In [30]:
# 'payment_pattern' 열에 대한 고유값 확인
unique_values = train['payment_pattern'].unique()

# 각 고유값에 대해 새로운 열 생성 및 0 또는 1로 설정
for value in unique_values:
    train[f'payment_pattern_{value}'] = (train['payment_pattern'] == value).astype(int)

# 기존 'payment_pattern' 열 삭제
train.drop('payment_pattern', axis=1, inplace=True)

for value in unique_values:
    test[f'payment_pattern_{value}'] = (test['payment_pattern'] == value).astype(int)

# 기존 'payment_pattern' 열 삭제
test.drop('payment_pattern', axis=1, inplace=True)

In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  int32  
 10  subscription_type                  10000 non-null  int32  
 11  customer_inquiry_history           10000 non-null  int6

In [32]:
columns_to_scale = train.columns.drop('target')  # 실제 열 이름으로 대체해야 함

# MinMaxScaler 객체 생성
scaler = MinMaxScaler()

# 선택한 열을 Min-Max Scaling
train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])
test[columns_to_scale] = scaler.fit_transform(test[columns_to_scale])

In [33]:
train_x = train.drop('target', axis = 1)
train_y = train['target']
test_x = test

In [34]:
ros = RandomOverSampler(random_state=42)
train_x, train_y = ros.fit_resample(train_x, train_y)

In [35]:
train_x

,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern_5,payment_pattern_6,payment_pattern_7,payment_pattern_0,payment_pattern_1,payment_pattern_3,payment_pattern_4,payment_pattern_2
0,0.545455,0.464286,0.510710,0.016719,0.739130,0.576923,0.422657,0.250000,0.75,0.0,0.0,0.4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.681818,0.607143,0.653087,0.144299,0.652174,0.461538,0.803441,0.166667,0.50,0.5,0.0,0.1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.954545,0.000000,0.561419,0.043235,0.521739,0.500000,0.761609,0.250000,0.75,0.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.642857,0.619612,0.084726,0.782609,0.653846,0.446031,0.000000,0.50,0.0,0.0,0.1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.136364,0.142857,0.772338,0.061055,0.782609,0.346154,0.599412,0.166667,0.75,0.5,0.0,0.3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12393,0.863636,0.035714,0.584262,0.248061,0.173913,0.346154,0.677915,0.083333,1.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
12394,0.545455,0.892857,0.439919,0.000164,0.869565,0.653846,0.477833,0.416667,1.00,0.0,1.0,0.3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12395,0.954545,0.142857,0.632324,0.023908,0.695652,0.384615,0.558721,0.166667,0.25,0.0,0.0,0.2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12396,0.409091,0.428571,0.603599,0.005290,0.130435,0.384615,0.379752,0.166667,0.50,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
train_y

0        0
1        1
2        1
3        1
4        1
        ..
12393    0
12394    0
12395    0
12396    0
12397    0
Name: target, Length: 12398, dtype: int64

In [37]:
train_x.fillna(0, inplace=True)
test_x.fillna(0, inplace=True)

In [106]:
import tensorflow as tf

def rmsle(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    squared_error = tf.square(tf.math.log1p(y_pred) - tf.math.log1p(y_true))

    return tf.sqrt(tf.reduce_mean(squared_error))

def loss_fn(y_true, y_pred):
    return rmsle(y_true, y_pred)

def metric_fn(y_true, y_pred):
    return rmsle(y_true, y_pred)

def f1_macro(y_true, y_pred):
    def precision_m(y_true, y_pred):
        true_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
        predicted_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
        return precision

    def recall_m(y_true, y_pred):
        true_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
        possible_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
        return recall

    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)

    f1 = 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))
    return f1

def binary_cross_entropy(y_true, y_pred):
    return tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)

# def f1_binary(y_true, y_pred):
#     def precision_m(y_true, y_pred):
#         true_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
#         predicted_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_pred, 0, 1)))
#         precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
#         return precision
# 
#     def recall_m(y_true, y_pred):
#         true_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
#         possible_positives = tf.reduce_sum(tf.round(tf.keras.backend.clip(y_true, 0, 1)))
#         recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
#         return recall
# 
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
# 
#     f1 = 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))
#     return f1

def f1_macro(y_true, y_pred):
    def recall_m(y_true, y_pred):
        true_positives = tf.math.count_nonzero(tf.math.round(y_true * y_pred), axis=0)
        possible_positives = tf.math.count_nonzero(tf.math.round(y_true), axis=0)
        recall = tf.math.divide_no_nan(true_positives, possible_positives)
        return recall

    def precision_m(y_true, y_pred):
        true_positives = tf.math.count_nonzero(tf.math.round(y_true * y_pred), axis=0)
        predicted_positives = tf.math.count_nonzero(tf.math.round(y_pred), axis=0)
        precision = tf.math.divide_no_nan(true_positives, predicted_positives)
        return precision

    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)

    f1 = tf.math.divide_no_nan(2 * (precision * recall), (precision + recall))
    f1_macro = tf.reduce_mean(f1)

    return f1_macro


In [135]:
callbacks_list = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=35, verbose=2, mode='min',restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=4, min_lr=0.000001),
    tf.keras.callbacks.TerminateOnNaN()
]

In [137]:
def create_model():

    input_layer = tf.keras.Input(shape=(len(train_x.columns), ))
    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    # tf.keras.layers.Dropout(0.1)
    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    # tf.keras.layers.Dropout(0.1)
    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    # tf.keras.layers.Dropout(0.2)
    
    # output_layer = tf.keras.layers.Dense(1)(x)
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                  loss=binary_cross_entropy,
                  metrics=['accuracy',f1_macro]
                  )

    return model

model = create_model()
history = model.fit(train_x.astype('float32'), train_y.astype('float32'),
                    epochs=300,
                    callbacks=callbacks_list,
                    validation_split=0.2)

Epoch 1/300
310/310 [==============================] - 2s 2ms/step - loss: 0.6770 - accuracy: 0.5904 - val_loss: 0.9374 - val_accuracy: 0.0617 - lr: 0.0010
Epoch 2/300
310/310 [==============================] - 1s 2ms/step - loss: 0.6611 - accuracy: 0.6061 - val_loss: 0.8626 - val_accuracy: 0.2194 - lr: 0.0010
Epoch 3/300
310/310 [==============================] - 1s 2ms/step - loss: 0.6528 - accuracy: 0.6158 - val_loss: 0.7846 - val_accuracy: 0.3153 - lr: 0.0010
Epoch 4/300
310/310 [==============================] - 1s 2ms/step - loss: 0.6491 - accuracy: 0.6234 - val_loss: 0.8744 - val_accuracy: 0.2048 - lr: 0.0010
Epoch 5/300
310/310 [==============================] - 1s 2ms/step - loss: 0.6447 - accuracy: 0.6195 - val_loss: 0.9010 - val_accuracy: 0.1391 - lr: 0.0010
Epoch 6/300
310/310 [==============================] - 1s 2ms/step - loss: 0.6424 - accuracy: 0.6270 - val_loss: 0.8229 - val_accuracy: 0.2597 - lr: 0.0010
Epoch 7/300
310/310 [==============================] - 1s 2ms/st

In [115]:
model.summary()

Model: "model_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_42 (InputLayer)       [(None, 20)]              0         
                                                                 
 batch_normalization_79 (Ba  (None, 20)                80        
 tchNormalization)                                               
                                                                 
 dense_160 (Dense)           (None, 32)                672       
                                                                 
 batch_normalization_80 (Ba  (None, 32)                128       
 tchNormalization)                                               
                                                                 
 dense_161 (Dense)           (None, 64)                2112      
                                                                 
 dense_162 (Dense)           (None, 128)               832

In [134]:
# 3405
sample_submission = pd.read_csv('../data/sample_submission.csv')
predictions = model.predict(test_x.astype('float32'))
binary_predictions = (predictions >= 0.5).astype(int)
sample_submission["target"] = binary_predictions
sample_submission.to_csv("../submission_data/9(64layer추가후매우낮아짐).csv", index=False)

313/313 [==============================] - 0s 901us/step


Epoch 1/100
310/310 [==============================] - 2s 2ms/step - loss: 0.6628 - accuracy: 0.6157 - f1_macro: 0.7554 - val_loss: 0.8934 - val_accuracy: 0.0234 - val_f1_macro: 0.0281 - lr: 0.0010
Epoch 2/100
220/310 [====================>.........] - ETA: 0s - loss: 0.6544 - accuracy: 0.6186 - f1_macro: 0.7603

KeyboardInterrupt: 

# Catboost

In [291]:
submit.to_csv("../submission_data/5.csv", index = False)

In [46]:
pred_final = model_xgb.predict(test)

In [47]:
submit = pd.DataFrame({
    'user_id': test_id,
    'target': pred_final.flatten()
})

In [48]:
submit.to_csv("../submission_data/5.csv", index = False)